<a href="https://colab.research.google.com/github/NPCA-TEAM/COVID-19/blob/main/Scripts/%206%20-%20AVALIA%C3%87%C3%83O_covid_ver2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparação do ambiente

## Login Drive

In [ ]:
#Inicia permitindo acesso ao Gdrive
from google.colab import drive
drive.mount('/content/drive')

## Instalações de dependências

In [ ]:
!pip install darts

In [ ]:
!pip install matplotlib==3.1.1

In [ ]:
!pip install pyyaml==5.4.1

## Importação de bibliotecas

In [ ]:
import matplotlib.backends

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import pandas as pd
import numpy as np
import glob
import os
import sys
import time
import torch
import math

import matplotlib.backends
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import matplotlib.dates as mdates
import seaborn as sns
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.gofplots import qqplot

from datetime import datetime
from darts import timeseries
from darts.dataprocessing.transformers import Scaler
from darts.models import KalmanFilter

from darts.metrics import mae, mse, mape, mase, smape, rmse, r2_score, coefficient_of_variation
from darts.utils.statistics import check_seasonality, plot_acf, plot_residuals_analysis, plot_hist
from darts.utils.timeseries_generation import datetime_attribute_timeseries

import warnings
warnings.filterwarnings('ignore')
import logging
logging.disable(logging.CRITICAL)
torch.manual_seed(1); np.random.seed(1)  # for reproducibility
import pickle

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.gofplots import qqplot
import statsmodels.api as sm

# Codificação para a avaliação do modelo

In [ ]:
import time
import timeit
from datetime import datetime

In [ ]:
datetime.today().strftime("%d_%m_%Y")

## Construção da classe para avaliação estatística e seleção do modelo (ajustar data na linha 8)

In [ ]:
class ErrorModel:

  def __init__(self, validation_or_train, sintoma_or_publicacao_or_ocorrencia, casos_obitos): # Inicialização
    self.casos_obitos = casos_obitos
    self.__validation_or_train = validation_or_train # Validação se é dados de validação ou treino
    self.pathErrorTable = f'/content/drive/MyDrive/NPCA - COVID/{self.casos_or_obitos_folder}/Errors' # Caminho dos erros
    self.models = ['NHITS','NBEATS', 'TFT', 'TCN', 'TRANSFORMER'] # Modelos utilizados 
    self.date = '28_11_2022' # Data --> Modificar para data atual  = MODIFICAR SEMPRE ESSA DATA
    self.sintoma_or_publicacao_or_ocorrencia = sintoma_or_publicacao_or_ocorrencia # Dados se forem de sintoma ou publicação
    self.path_seletedModels = f'/content/drive/MyDrive/NPCA - COVID/{self.casos_or_obitos_folder}/SelectedModel/' # Caminho dos modelos selecionados
    self.TableError()
# ========================================================================================
# ------------------------------------- Setters ------------------------------------------
# ========================================================================================
  
  

  @property
  def casos_or_obitos_folder(self):
    if self.casos_obitos == 'casos':
      self.casos_or_obitos = '_CASOS'
    elif self.casos_obitos == 'obitos':
      self.casos_or_obitos = '_OBITOS'
   
    return self.casos_or_obitos

  @property
  def sintoma_or_publicacao_or_ocorrencia_column(self):
    if self.casos_obitos == 'casos':
      if self.sintoma_or_publicacao_or_ocorrencia == 'sintoma': # Pegando os dados de sintoma ou publicação
        column = 'DataSintoma'
      elif self.sintoma_or_publicacao_or_ocorrencia == 'publicacao':
        column = 'Publicacao'
    elif self.casos_obitos == 'obitos': 
      if self.sintoma_or_publicacao_or_ocorrencia == 'ocorrencia': # Pegando os dados de sintoma ou publicação
        column = 'DataOcorrencia'
      elif self.sintoma_or_publicacao_or_ocorrencia == 'publicacao':
        column = 'Publicacao'
    return column

  @property
  def casos_obitos_column(self):
    if self.casos_obitos == 'casos':
      first = 'Casos'
    elif self.casos_obitos == 'obitos':
      first = 'Obitos'

    return first
 

  @property
  def real_model(self): # Extraindo os dados reais
    
    path_dataset = f'/content/drive/MyDrive/NPCA - COVID/{self.casos_or_obitos_folder}/DataSet/'  
    file_list = glob.glob(path_dataset + '*')
    file_path = max(file_list, key=os.path.getctime)

    dataframe = pd.read_excel(file_path, sheet_name=0)
    
    dataframe = dataframe.loc[:, ['data', f'{self.casos_obitos_column}_{self.sintoma_or_publicacao_or_ocorrencia_column}_MM_atual_PA']]

    dataframe[f'{self.casos_obitos_column}_{self.sintoma_or_publicacao_or_ocorrencia_column}_MM_atual_PA'] = np.where(dataframe[f'{self.casos_obitos_column}_{self.sintoma_or_publicacao_or_ocorrencia_column}_MM_atual_PA'] <= 0, 0.00001, dataframe[f'{self.casos_obitos_column}_{self.sintoma_or_publicacao_or_ocorrencia_column}_MM_atual_PA'])
    
    t_s = timeseries.TimeSeries.from_dataframe(df=dataframe, time_col= 'data')
    t_s = t_s.astype('float64')

    return t_s
    

  @property
  def read_and_process_models(self): # Leitura e processamento dos conjuntos de dados
   
    if self.__validation_or_train == 'train':
      path_forecasts = "training_Forecast.csv"

    elif self.__validation_or_train == 'validation':
      path_forecasts = "validation_Forecast.csv"

    path_NHITS = f'/content/drive/MyDrive/NPCA - COVID/{self.casos_or_obitos_folder}/ForecastsOutputModel/NHITS-Model_{self.getDate}/{path_forecasts}'
    path_NBEATS = f'/content/drive/MyDrive/NPCA - COVID/{self.casos_or_obitos_folder}/ForecastsOutputModel/NBEATS-Model_{self.getDate}/{path_forecasts}'
    path_TFT = f'/content/drive/MyDrive/NPCA - COVID/{self.casos_or_obitos_folder}/ForecastsOutputModel/TFT-Model_{self.getDate}/{path_forecasts}'
    path_TNC = f'/content/drive/MyDrive/NPCA - COVID/{self.casos_or_obitos_folder}/ForecastsOutputModel/TCN-Model_{self.getDate}/{path_forecasts}'
    path_Tranformer = f'/content/drive/MyDrive/NPCA - COVID/{self.casos_or_obitos_folder}/ForecastsOutputModel/TRANSFORMER-Model_{self.getDate}/{path_forecasts}'

    
    # Cria DF
    df_NHITS = pd.read_csv(path_NHITS)
    df_NBEATS = pd.read_csv(path_NBEATS)
    df_TFT = pd.read_csv(path_TFT)
    df_TNC = pd.read_csv(path_TNC)
    df_Tranformer = pd.read_csv(path_Tranformer)

    # Pós Processamento
    # Valores < 0 = 0

    df_de_modelos =  [df_NHITS, df_NBEATS, df_TFT, df_TNC, df_Tranformer]

    series_forecast = []

    # Pega colunas Data e Casos de sintormas e publicação
    
    for i in df_de_modelos:
      valForecasts = i.loc[:, ['data', f'{self.casos_obitos_column}_{self.sintoma_or_publicacao_or_ocorrencia_column}_MM_atual_PA']]
      valForecasts[f'{self.casos_obitos_column}_{self.sintoma_or_publicacao_or_ocorrencia_column}_MM_atual_PA'] = np.where(valForecasts[f'{self.casos_obitos_column}_{self.sintoma_or_publicacao_or_ocorrencia_column}_MM_atual_PA'] <= 0, 0.00001, valForecasts[f'{self.casos_obitos_column}_{self.sintoma_or_publicacao_or_ocorrencia_column}_MM_atual_PA'])

      serie_valForecasts = timeseries.TimeSeries.from_dataframe(df=valForecasts,  time_col= 'data')
      series_forecast.append(serie_valForecasts)
      


    return series_forecast


# ========================================================================================
# ----------------------------------- Getters --------------------------------------------
# ========================================================================================

  @property
  def getSintomaPublicacao(self): # Get se os dados são de sintoma ou publicação
    return self.sintoma_or_publicacao_or_ocorrencia
  @property
  def getValidationOrTrain(self): # Get se os dados são de validação ou treino
    return self.__validation_or_train 
  @property
  def getBestModel(self): # Get o melhor modelo do conjunto de dados
    table_error = self.TableError()
    return table_error.loc[table_error['NOTA_GERAL'] == table_error['NOTA_GERAL'].max()]['MODELO'].values[0]
  @property
  def getDate(self): # Get data dos dados (mesma para reais e forecast)
    return self.date
  
  def getGlobalBestModelPath(self, n_models): # Get caminho do melhor modelo
    bestmodels_g_path_r = []
    bestmodels_g_path_e = []

    for i in self.get_general_best_model(n_models)['Residuos']:
      globalBestModelPath = f'/content/drive/MyDrive/NPCA - COVID/{self.casos_or_obitos}/Models/{i}-Model_{self.getDate}.pth.tar'
      bestmodels_g_path_r.append(globalBestModelPath)
    for i in self.get_general_best_model(n_models)['Erros']:
      globalBestModelPath = f'/content/drive/MyDrive/NPCA - COVID/{self.casos_or_obitos}/Models/{i}-Model_{self.getDate}.pth.tar'
      bestmodels_g_path_e.append(globalBestModelPath)

    globalBestModelPath = {}
    globalBestModelPath['Residuos'] = bestmodels_g_path_r
    globalBestModelPath['Erros'] = bestmodels_g_path_e
    globalBestModelPath['Score'] = list(range(len(bestmodels_g_path_e), 0, -1))
    globalBestModelPath = pd.DataFrame(globalBestModelPath)

    return globalBestModelPath


# ========================================================================================
# ----------------------------------- Tabela de erros ------------------------------------
# ========================================================================================

  def TableError(self): 
    values = []
   
    for j, k in zip(self.read_and_process_models, self.models): # Calculando erro caso os dados fonecidos sejam uma lista
     
      metric_mse = mse(self.real_model, j) 
      metric_mape = mape(self.real_model, j)  
      metric_smape = smape(self.real_model, j) 
      metric_rmse = rmse(self.real_model, j) 
      #print(self.real_model[-8:])
      #print(j)    
      metric_r2_score = r2_score(self.real_model, j)  

      if math.isnan (metric_r2_score) or math.isinf (metric_r2_score):	
        metric_r2_score = 0.0
        #print('oi')
      metric_coefficient_of_variation = coefficient_of_variation(self.real_model, j) 
      values.append([metric_mse, metric_mape, metric_smape, metric_rmse, metric_r2_score, metric_coefficient_of_variation, k.upper()]) 


    df = pd.DataFrame(columns = ['MSE', 'MAPE', 'SMAPE', 'RMSE', 'R2_SCORE', 'COEF_OF_VARIATION', 'MODELO']) # Criando df vazio para anexar notas

    for i in values:
      result = dict(zip(['MSE', 'MAPE', 'SMAPE', 'RMSE', 'R2_SCORE', 'COEF_OF_VARIATION', 'MODELO'], i)) # Criando dicionário com resultados
      df_to_append = pd.DataFrame(result, index = [datetime.today().strftime("%d-%m-%Y")]) # Criando DF com resultados
      df = pd.concat([df, df_to_append]) 
      
    tamanho = df.shape[0] # Buscando tamanho do dataframe para dar as notas 
    notas = np.linspace(0, 1, tamanho) # Definindo a maior nota como 1, menor 0 e o espaçamento dependete da quantidade de registros
    new_df = df[['MODELO']]
    new_df = new_df.sort_values(by = 'MODELO', ascending = True) # Criando dataframe somente com o nome dos modelos
    metricas_df = df[['MODELO']].sort_values(by = 'MODELO', ascending = True) # Criando dataframe somente com o nome dos modelos
    

    for i in df.columns:     
      if i in ['MSE', 'MAPE', 'SMAPE', 'RMSE', 'COEF_OF_VARIATION']: # Quanto menor, melhor
        df[[i, 'MODELO']] = df[[i, 'MODELO']].sort_values(by= i, ascending = False)
        metricas_df = metricas_df.merge(df[[i, 'MODELO']] , on = ['MODELO'])
        df[i] = notas
        new_df = new_df.merge(df[[i, 'MODELO']], how = 'inner', on = ['MODELO'])

        # Juntando Serie de notas e utilizando o nome do modelo, para atribuir corretamente
      elif i == 'MODELO': # Ignorar a coluna MODELO
        continue 
      elif i in ['R2_SCORE']: # Quanto maior, melhor - VERIFICAR 
        df[[i, 'MODELO']] = df[[i, 'MODELO']].sort_values(by= i, ascending = True)
        metricas_df = metricas_df.merge(df[[i, 'MODELO']] , on = ['MODELO'])
        df[i] = notas
        new_df = new_df.merge(df[[i, 'MODELO']], how = 'inner', on = ['MODELO'])

    new_df['NOTA_GERAL'] = new_df[['MSE', 'MAPE', 'SMAPE', 'R2_SCORE', 'COEF_OF_VARIATION']].sum(axis=1, numeric_only=True) # Extraindo a soma das notas fornecidas
    metricas_df = metricas_df.sort_values(by= 'MODELO', ascending = True)
    new_df = new_df.sort_values(by= 'MODELO', ascending = True)
    metricas_df['NOTA_GERAL'] = new_df['NOTA_GERAL']

    return metricas_df

# ========================================================================================
# -------------------------------- Getters com argumentos --------------------------------
# ----------------------------------- Tabela de erros ------------------------------------
# ========================================================================================


  def getSeriesReais(self, format): # Extraindo dados reais -- Retorna em DataFrame do Pandas ou em timeseries do darts
    if format == 'dataframe':
      return self.real_model.pd_dataframe()
    elif format == 'timeseries':
      return self.real_model

    
  def getSeriesForecast(self, format): # Extraindo dados forecast -- Retorna em DataFrame do Pandas ou em timeseries do darts
    if format not in ['dataframe', 'timeseries']:
      raise TypeError('Os formatos suportados são dataframe ou timeseries, em vez disso foi passado {}'.format(format))
    elif format == 'dataframe':
      dataframes = []
      for i in self.read_and_process_models:
        dataframe = i.pd_dataframe()
        dataframes.append(dataframe)
      return dict(zip(self.models, dataframes))
    elif format == 'timeseries':
      self.read_and_process_models


# ========================================================================================
# --------------------------------------- Funções ----------------------------------------
# ----------------------------------- Tabela de erros ------------------------------------
# ========================================================================================


  def showErrorTable(self): # Mostra tabela de erros
    return self.TableError()

  def get_n_BestModel_errors(self, n): # Extraindo o n melhores modelos 
    table_error = self.TableError()
    if n > len(table_error['MODELO'].values[:n]) +1: # Caso o argumento seja maior que a quantidade de dados
      raise IndexError('O valor é maior que o conjunto de dados')
    else:
      return table_error.sort_values(by='NOTA_GERAL', ascending=False)['MODELO'].values[:n]

  def saveBestModelPath(self, n): # Salvando tabela com o melhor modelo.
    path = f'{self.pathErrorTable}/PathBestModels/PathBestModels_{self.getValidationOrTrain}_{self.getSintomaPublicacao}_{self.getDate}.csv'
    self.getGlobalBestModelPath(n).to_csv(path, index = False)
    print('Best Models Table saved at:\t {}'.format(path))
    return path

  def saveErrorTable(self): # Salvando tabela de erros
    
    path = f'{self.pathErrorTable}/ErrorsTable/ErrorTable_{self.getValidationOrTrain}_{self.getSintomaPublicacao}_{self.getDate}.csv'
    self.TableError().to_csv(path, index = False)
    print(f'Error Table saved at:\t{path}')
    return path

  def saveAll(self, n): # Salvando tudo.
    self.saveBestModelPath(n)
    self.saveErrorTable()
    self.saveResiduos()
    # self.saveAllFigures(n)

# ========================================================================================
# --------------------------------------- Funções ----------------------------------------
# --------------------------------------- Residuos ---------------------------------------
# ========================================================================================
 
  def gerarResiduos(self): # Gerando dicionário com os resíduos
    dict_residuos = {}
    for i, j in  self.getSeriesForecast(format='dataframe').items():
      residuos = self.getSeriesForecast(format='dataframe')[i].merge(self.getSeriesReais(format = 'dataframe'), 'inner', on ='data')
      residuos = residuos[f'{self.casos_obitos_column}_{self.sintoma_or_publicacao_or_ocorrencia_column}_MM_atual_PA_y'] - residuos[f'{self.casos_obitos_column}_{self.sintoma_or_publicacao_or_ocorrencia_column}_MM_atual_PA_x']

      dict_residuos[i] = residuos

    
    return pd.DataFrame(dict_residuos)

  def plotResiduos(self, n_best_models=None):
    figs = []
    models = []
    if n_best_models == None:
      n_best_models = 5
    best_models = self.get_n_BestModel_errors(n_best_models)
    for i in best_models:
      fig, ax = plt.subplots(2,2,figsize=(16,12))
      fig.suptitle(f'{i}\n{self.casos_obitos}, {self.sintoma_or_publicacao_or_ocorrencia}, {self.getValidationOrTrain}\n{self.getDate}\n\n')
      residuos = self.gerarResiduos()[i]
      ax_l = qqplot(residuos, ax =ax[1][1], line = 's')
      #plt.text(-0.35, 3500, 'QQ Plot', fontdict={'size':12})
      ax_d = sns.distplot(residuos,ax=ax[0][1], bins =7)
      ax_d.set_ylabel('Values')
      ax_d.set_title('Distribution')
      ax_d.set_xlabel('Count')
      ax_acf = plot_acf(residuos, ax=ax[1][0], title = 'AutoCorrelation Function (ACF)')
      font ={'weight':'bold'}
      #plt.text(-8,-2100, 'Lag', fontdict = font)
      #plt.text(-11.49,1000, 'ACF Value', fontdict = font, rotation = 90)
      x = list(range(0,len(residuos)))
      ax_reg = sns.regplot(x,residuos,label = 'Best-fit line',scatter_kws={"s": 10}, ax = ax[0][0], line_kws={'color':'red'})
      new_x = list(self.gerarResiduos().index.astype('str'))
      ax_reg.set_xticks(x, new_x)
      ax_reg.set_title('Residual Values')
      ax_reg.set_xlabel('Date')
      ax_reg.set_ylabel('Residual values')
      plt.show()

      print('-------------------------')
      fig.savefig(f'/content/drive/MyDrive/NPCA - COVID/PlotsTemp/{str(datetime.now())}.png')#deletar depois de ajustado
      print('-------------------------')
      figs.append(fig)
      models.append(i)
    models = dict(zip(models, figs))
    return models
  
  def saveAllFigures(self, n):
    ax = self.plotResiduos(n)
    for model in ax:
      ax[model].savefig(f'{self.pathErrorTable}/Residuos/Plots/Plot-{model}_{self.getSintomaPublicacao}_{self.getDate}.png')
  
  def saveFigure(self, model):
    ax = self.plotResiduos(5)
    ax[model].savefig(f'{self.pathErrorTable}/Residuos/Plots/Plot-{model}_{self.getSintomaPublicacao}_{self.getValidationOrTrain}_{self.getDate}.png');

  def showResiduosInfo(self):
    columns = ['COEF_ANGULAR', 'MEAN', 'STD', 'MODELO']
    df = pd.DataFrame(columns = columns)
    for i in self.models:
      residuos = self.gerarResiduos()[i]
      x = list(range(0,len(residuos)))
      reg_model = sm.OLS(residuos, x).fit()

      coef_angular = reg_model.params.x1
      mean = residuos.mean()
      std = residuos.std()

      df2 = pd.DataFrame(dict(zip(columns, [coef_angular, mean, std, i])), index = [i])
      df = pd.concat([df, df2])
    notas = np.linspace(0, 1, df.shape[0])
    df['NOTA_GERAL'] = 0
    for i in  ['COEF_ANGULAR', 'MEAN', 'STD']:
      df_new = df[['MODELO', i]]
      df_new[i] = abs(df_new[i])
      df_new = df_new.sort_values(by= i, ascending = False)
      df = df.sort_values(by= i, ascending = False)
      df_new[i] = notas
      df['NOTA_GERAL'] = df['NOTA_GERAL'] + df_new[i]
    df = df[['MODELO', 'COEF_ANGULAR', 'MEAN', 'STD', 'NOTA_GERAL']]
    df = df.reset_index(drop = True)
    return df

  def get_n_Best_Residuos(self, n):
    best_residuos = self.showResiduosInfo().sort_values(by='NOTA_GERAL', ascending=False)['MODELO'].values[:n]
    return best_residuos

  def saveResiduos(self):
    path = f'{self.pathErrorTable}/Residuos/Tables/ResiduoInfo_{self.getValidationOrTrain}_{self.getSintomaPublicacao}_{self.getDate}.csv'
    self.showResiduosInfo().to_csv(path, index = False)
    print(f'Residual table saved at: {path}')
# ========================================================================================
# --------------------------------------- Funções ----------------------------------------
# ---------------------------------- Residuos & Erros ------------------------------------
# ========================================================================================

  def get_general_best_model(self, n, format='dict'):
    a = self.get_n_Best_Residuos(n)
    b = self.get_n_BestModel_errors(n)

    if format == 'dict':
      best_G_model = {'Residuos':a, 'Erros':b}
      return best_G_model
    elif format == 'list':
      a = list(a)
      b = list(b)
      for i in b:
        a.append(i)
      return a
  

  def compare_with(self, other_model, save_best_path = False, save_plot = False):
    dict_best = {}
    model_a = self.get_general_best_model(2, format = 'list')
    model_b = other_model.get_general_best_model(2, format = 'list')
    for i in model_b:
      model_a.append(i)
    for i in model_a:
      dict_best[i] = model_a.count(i)
    
    if save_best_path:
      columns = ['model', 'path', 'data']
      best_compare_model = f'/content/drive/MyDrive/NPCA - COVID/{self.casos_or_obitos_folder}/Models/{max(dict_best, key=dict_best.get)}-Model_{self.getDate}.pth.tar'
      pd.DataFrame(dict(zip(columns, [max(dict_best, key=dict_best.get), best_compare_model, self.getDate])),index= [0]).to_csv(f'/content/drive/MyDrive/NPCA - COVID/{self.casos_or_obitos_folder}/SelectedModel/PathBestModels_{self.sintoma_or_publicacao_or_ocorrencia}_{self.getDate}.csv', index= False)
    if save_plot:
      print(dict_best)
      self.saveFigure(max(dict_best, key=dict_best.get))
    return dict_best

In [ ]:
#ErrorModel('train', 'sintoma', 'casos').gerarResiduos().index[800]

In [ ]:
#ErrorModel('train', 'sintoma', 'casos').plotResiduos(1)

##Avaliação dos modelos ajustados/treinados

In [ ]:
errosSintoma_TRAIN_CASOS = ErrorModel('train', 'sintoma', 'casos')
errosSintoma_VAL_CASOS = ErrorModel('validation', 'sintoma', 'casos')

errosSintoma_TRAIN_CASOS.saveAll(5)
errosSintoma_VAL_CASOS.saveAll(5)

errosSintoma_VAL_CASOS.compare_with(errosSintoma_TRAIN_CASOS, True, True)

In [ ]:
errosPublicacao_TRAIN_CASOS =  ErrorModel('train', 'publicacao', 'casos')
errosPublicacao_VAL_CASOS =  ErrorModel('validation', 'publicacao', 'casos')

errosPublicacao_TRAIN_CASOS.saveAll(5)
errosPublicacao_VAL_CASOS.saveAll(5)

errosPublicacao_TRAIN_CASOS.compare_with(errosPublicacao_VAL_CASOS, True, True)

In [ ]:
errosPublicacao_TRAIN_OBITOS =  ErrorModel('train', 'publicacao', 'obitos')
errosPublicacao_VAL_OBITOS =  ErrorModel('validation', 'publicacao', 'obitos')

errosPublicacao_TRAIN_OBITOS.saveAll(5)
errosPublicacao_VAL_OBITOS.saveAll(5)

errosPublicacao_TRAIN_OBITOS.compare_with(errosPublicacao_VAL_OBITOS, True, True)

In [ ]:
errosOcorrencia_TRAIN_OBITOS =  ErrorModel('train', 'ocorrencia', 'obitos')
errosOcorrencia_VAL_OBITOS =  ErrorModel('validation', 'ocorrencia', 'obitos')

errosOcorrencia_TRAIN_OBITOS.saveAll(5)
errosOcorrencia_VAL_OBITOS.saveAll(5)

errosOcorrencia_TRAIN_OBITOS.compare_with(errosOcorrencia_VAL_OBITOS, True, True)

# Plots

## Casos

### Plot de resíduos de sitomas de treinamento para casos

In [ ]:
errosSintoma_TRAIN_CASOS.plotResiduos()

### Plot de resíduos de sitomas de validação para casos

In [ ]:
errosSintoma_VAL_CASOS.plotResiduos()

### Plot de resíduos de publicação de treinamento para casos

In [ ]:
errosPublicacao_TRAIN_CASOS.plotResiduos()

### Plot de resíduos de publicação de validação para casos

In [ ]:
errosPublicacao_VAL_CASOS.plotResiduos()

## Obitos

### Plot de resíduos de publicação de treinamento para obitos

In [ ]:
errosPublicacao_TRAIN_OBITOS.plotResiduos()

### Plot de resíduos de publicação de validação para obitos

In [ ]:
errosPublicacao_VAL_OBITOS.plotResiduos()

### Plot de resíduos de ocorrência de treinamento para obitos

In [ ]:
errosOcorrencia_TRAIN_OBITOS.plotResiduos()

### Plot de resíduos de ocorrência de validação para obitos

In [ ]:
errosOcorrencia_VAL_OBITOS.plotResiduos()

# Tabelas de erros

In [ ]:
errosOcorrencia_TRAIN_OBITOS.showErrorTable()

In [ ]:
errosOcorrencia_VAL_OBITOS.showErrorTable()

In [ ]:
errosPublicacao_VAL_OBITOS.showErrorTable()

In [ ]:
errosPublicacao_TRAIN_OBITOS.showErrorTable()

In [ ]:
errosPublicacao_VAL_CASOS.showErrorTable()

In [ ]:
errosPublicacao_TRAIN_CASOS.showErrorTable()

In [ ]:
errosSintoma_TRAIN_CASOS.showErrorTable()

In [ ]:
errosSintoma_VAL_CASOS.showErrorTable()

#Tabela de Resíduos